## Resnet18

In [2]:
import torch
from torch2trt import TRTModule
import cv2
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
device = torch.device('cuda')
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()


# 載入 ResNet 模型
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt2.pth'))

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]


def resnet(image, height):
    # 裁剪影像
    cropped_image = image[int(height * 0.4):, :, :]

    # 模型推理，確保使用 no_grad
    with torch.no_grad():
        # 預處理
        input_data = preprocess(cropped_image).cuda()
        # 推理
        xy = model_trt(input_data).detach().float().cpu().numpy().flatten()
        torch.cuda.synchronize()  # 確保推理完成
    x = xy[0]
    y = xy[1]
    return x, y
    

## 測試Resnet18


In [ ]:
# 測試resnet
if __name__ == "__main__":
    image_path = 'test2.jpg'
    image = cv2.imread(image_path)
    print(image_path)
    if image is None:
        print(f"Error: Unable to load image from {image_path}")
    else:
        # 將圖像轉換為 NumPy 數組
        image_array = np.array(image)
        print(f"Image successfully loaded and converted to NumPy array. Shape: {image_array.shape}")

        height = image_array.shape[0]

        # ResNet 推理
        x, y = resnet(image_array, height)
        if x is not None and y is not None:
            print(f"ResNet prediction: x={x}, y={y}")
        else:
            print("ResNet inference failed.")


## 準備


In [3]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera(width=224, height=224,fps=9)


In [15]:
import requests
import ipywidgets.widgets as widgets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg

image_widget = widgets.Image(
    format='jpg',
    width=300,  # 设置图像宽度
    height=400,  # 设置图像高度
)

# 创建标签小部件
label_widget = widgets.Label(
    value="等待路牌資料中~",  # 设置标签文本
    style={'font_size': '50px', 'color': 'blue'}  # 设置标签样式
)
label_widget2 = widgets.Label(
    value="等待循線資料中~",  # 设置标签文本
    style={'font_size': '50px', 'color': 'black'}  # 设置标签样式
)
# 将图像和标签小部件垂直排列
vbox = widgets.VBox([image_widget, label_widget,label_widget2])

# 显示组合的小部件
display(vbox)

#-----------設定參數--------------#
robot = Robot()
robot.motor_driver._pwm.setPWMFreq(80)
angle = 0.0
angle_last = 0.0
integral = 0.0  # 初始化積分項
left_motor_value,right_motor_value = 0,0
no_move_inter = 0
signs = []
previous_mse = None
last_execution_time = 0
start_time = time.time()


last_triggered = {
    "Stop": 0,
    "Stop and Proceed": 0,
    "Railroad Crossing Warning": 0,
    'Maximum Speed Limit':0,
    'Minimum Speed Limit':0,
    'Pedestrian Crossing':0,
}
trigger_interval = 10
def analyze_image(image_array):
    # 将 NumPy 数组转换为 JPEG 格式
    _, image_encoded = cv2.imencode('.jpeg', image_array)
    image_bytes = image_encoded.tobytes()  # 转换为字节格式

    # 将字节作为文件上传
    response = requests.post(
        "http://0.0.0.0:5000/analyze-image",
        files={"file": ("image.jpeg", image_bytes, "image/jpeg")}
    )
    return response.json()  # 假设 API 返回 JSON 格式的分析结果
def update(change):
    global no_move_inter,previous_mse
    global angle, angle_last,last_triggered
    global last_execution_time,start_time
    
    if time.time()>start_time:
        label_widget2.value = "動作完成中"
        start_time = time.perf_counter()

        #-----------不動加速圖片--------------#
        image_array = change['new']
        old_image =  change['old']
        mse = np.sum((image_array - old_image) ** 2) / float(image_array.size)
        if mse < 50:
            if previous_mse is not None and previous_mse < 50:
                no_move_inter += 1  # 保持累加
                no_move_inter = min(no_move_inter,200)
            else:
                no_move_inter = 1   # 重設或初始化
        elif mse > 90:
            no_move_inter = -300
        else:
            no_move_inter = 0
        previous_mse = mse


        #-----------顯示圖片--------------#
        #_, encoded_image = cv2.imencode('.jpeg', image_array)
        #image_widget.value = encoded_image.tobytes()

        #-----------道路追蹤--------------#
        height = image_array.shape[0]
        x, y = resnet(image_array, height)
        x_slider.value = x
        y_slider.value = y
        
        y = y * 0.7
        angle = np.arctan2(x, y)

        #-----------PID 控制--------------#
        
        
        if angle < 0:
            pid = (
                angle * steering_gain_slider.value  # P 項
                + (angle - angle_last) * steering_dgain_slider.value  # D 項
            )
            steering_slider.value = pid + steering_left_bias_slider.value  # 如果 angle < 0，添加偏置
        else:
            pid = (
                angle * (steering_gain_slider.value-0.01)  # P 項
                + (angle - angle_last) * (steering_dgain_slider.value+0.01)  # D 項
            )
            steering_slider.value = pid + steering_right_bias_slider.value  # 如果 angle >= 0，減去偏置
        
        angle_last = angle  # 更新上一個角度
        
        # 計算馬達值
        speed_slider.value = speed_gain_slider.value+(no_move_inter//10)*0.001
        right_motor_value = -max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
        left_motor_value = -max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        #print(speed_gain_slider.value)


        #-----------偵測路牌--------------#
        signs = []
        current_time = time.time()
        if current_time - last_execution_time >= 1:
            signs = analyze_image(image_array)
            print("API 結果:", signs)
            # 更新上次執行時間
            last_execution_time = current_time
            
        target = None
        if len(signs)>0:
            target = signs[0]
            if  target[0]>50:
                label_widget.value = target
            else:
                label_widget.value = f"目前無偵測到路牌"

        else:
             label_widget.value = f"目前無偵測到路牌"


        if target is None :#沒有路牌
            robot.left_motor.value = left_motor_value
            robot.right_motor.value = right_motor_value
            left_motor_slider.value = left_motor_value
            right_motor_slider.value = right_motor_value
        else:
            print(target)
            if target[0]>10:
                #-----------下個路牌--------------#
                if target=="Stop" :
                    current_time = time.time()
                    if current_time - last_triggered["Stop"] > trigger_interval:
                        robot.left_motor.value = 0
                        robot.right_motor.value = 0
                        left_motor_slider.value = 0
                        right_motor_slider.value = 0
                        label_widget2.value = "直接停下來"
                        last_triggered["Stop"] = current_time
                        start_time = current_time+100
                elif target == "Stop and Proceed":
                    current_time = time.time()
                    if current_time - last_triggered["Stop and Proceed"] > trigger_interval:
                        robot.left_motor.value = 0
                        robot.right_motor.value = 0
                        left_motor_slider.value = 0
                        right_motor_slider.value = 0
                        label_widget2.value = "正在執行 等待兩秒"
                        start_time = current_time+2
                        last_triggered["Stop and Proceed"] = current_time
                        

                elif target == "Railroad Crossing Warning":
                    current_time = time.time()
                    if current_time - last_triggered["Railroad Crossing Warning"] > trigger_interval:
                        robot.left_motor.value = 0
                        robot.right_motor.value = 0
                        left_motor_slider.value = 0
                        right_motor_slider.value = 0
                        label_widget2.value = "正在執行 等待五秒"
                        start_time = current_time+5
                        last_triggered["Railroad Crossing Warning"] = current_time
                else:# 間隔時間不夠
                    robot.left_motor.value = left_motor_value
                    robot.right_motor.value = right_motor_value
                    left_motor_slider.value = left_motor_value
                    right_motor_slider.value = right_motor_value
            else:#路牌不夠大
                robot.left_motor.value = left_motor_value
                robot.right_motor.value = right_motor_value
                left_motor_slider.value = left_motor_value
                right_motor_slider.value = right_motor_value
        end_time = time.perf_counter()
        # 計算執行時間（秒）
        execution_time = end_time - start_time
        label_widget.value = label_widget.value + f" 程式執行時間: {execution_time:.4f} 秒"  
        

In [5]:
import ipywidgets.widgets as widgets
import ipywidgets as widgets
from IPython.display import display
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.02, description='steering kd')
steering_left_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias left')
steering_right_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias right')
steering_igain_slider = widgets.FloatSlider(
    min=0.0, max=1.0, step=0.01, value=0.00, description='Steering I Gain'
)
left_motor_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, description='Left Motor')
right_motor_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, description='Right Motor')
angle_slider = widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.01, description='Angle')
display(left_motor_slider, right_motor_slider, angle_slider)
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_left_bias_slider,steering_right_bias_slider,steering_igain_slider)

FloatSlider(value=0.0, description='Left Motor', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.0, description='Right Motor', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.0, description='Angle', max=3.141592653589793, min=-3.141592653589793, step=0.01)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.1, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.02, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias left', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='steering bias right', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='Steering I Gain', max=1.0, step=0.01)

In [6]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.01 ,description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, step=0.001, orientation='vertical', description='speed')
display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0, step=0.01)

In [ ]:
import time
camera.observe(update, names='value')

In [ ]:
camera.stop()

In [ ]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()